In [129]:
#library and data imports
import pandas as pd

election_data = pd.read_csv('election_train.csv')
demographics_data = pd.read_csv('demographics_train.csv')


#print(election_data.head())
#print(demographics_data.head())



In [82]:
#Task 1: Reshaping election_train into wide format

# Reshaping the data by pivoting along the County and unstacking the Party column of the original data set
reshaped_data = election_data.pivot_table(index=['County', 'State', 'Year', 'Office'], values='Votes',columns='Party' )

reshaped_data.reset_index(inplace=True) # we must reshape the index to get rid of multi-level pivot table

reshaped_data.columns.name = None # here we get rid of the index column name

print(reshaped_data)




             County State  Year      Office  Democratic  Republican
0      Adams County    IN  2018  US Senator      3146.0      7511.0
1      Adams County    ND  2018  US Senator       364.0       796.0
2      Adams County    NE  2018  US Senator      3334.0      6487.0
3      Adams County    OH  2018  US Senator      2635.0      6000.0
4      Adams County    PA  2018  US Senator     14880.0     23419.0
...             ...   ...   ...         ...         ...         ...
1200    York County    ME  2018  US Senator     51387.0     32849.0
1201    York County    NE  2018  US Senator      1281.0      3659.0
1202    York County    PA  2018  US Senator     69272.0     95814.0
1203   Young County    TX  2018  US Senator       821.0      5543.0
1204  Zapata County    TX  2018  US Senator      1392.0       821.0

[1205 rows x 6 columns]


In [189]:
#Task 2: Merging reshaped election data with demographics data

# dictionary to convert state names to abbreviations, sourced from github.com/rogerallen
state_dict = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

"""
Fixing up the election data

"""

# handling inconsistencies in the names of counties
reshaped_data['County'] = reshaped_data['County'].str.replace('County', '')
reshaped_data['County'] = reshaped_data['County'].str.strip();

# removing rows where there is null republican or democratic data
reshaped_data = reshaped_data.dropna()

"""
Fixing up the demographics data

"""

# replacing state names with abbreivations
demographics_data = demographics_data.replace({"State": state_dict})
demographics_data = demographics_data.sort_values(['County', 'State'])

# resetting the indexs after sorting to match election data
demographics_data = demographics_data.reset_index(drop=True)



# doing a left join on the data results in a dataset having rows where the demographics data matches one of our 1205 rows
merged_data = pd.merge(reshaped_data, demographics_data, on=['State', 'County'], how='left')

#print(reshaped_data.head())
#print(demographics_data.head())
#rslt_df = merged_data[merged_data['_merge'] != 'both'] 
#rslt_df = merged_data[merged_data['Republican'] < 1.0] 
#print(rslt_df)
# print(merged_data['Republican'].isnull().sum())
# print(merged_data['Democratic'].isnull().sum())

print(merged_data.iloc[:, 0:9]) #not printing all columns for cleaner output


      County State  Year      Office  Democratic  Republican     FIPS  \
0      Adams    IN  2018  US Senator      3146.0      7511.0  18001.0   
1      Adams    ND  2018  US Senator       364.0       796.0  38001.0   
2      Adams    NE  2018  US Senator      3334.0      6487.0  31001.0   
3      Adams    OH  2018  US Senator      2635.0      6000.0  39001.0   
4      Adams    PA  2018  US Senator     14880.0     23419.0  42001.0   
...      ...   ...   ...         ...         ...         ...      ...   
1195    York    ME  2018  US Senator     51387.0     32849.0  23031.0   
1196    York    NE  2018  US Senator      1281.0      3659.0  31185.0   
1197    York    PA  2018  US Senator     69272.0     95814.0  42133.0   
1198   Young    TX  2018  US Senator       821.0      5543.0  48503.0   
1199  Zapata    TX  2018  US Senator      1392.0       821.0  48505.0   

      Total Population  Citizen Voting-Age Population  
0              34813.0                            0.0  
1          

In [190]:
#Task 3: Exploring the merged data

# printing out the shape
print("Size of dataset: ", merged_data.shape)
print("*****")

# displaying the info for the columns
merged_data.info()
print("*****")

# inquiring about redundant and irrelevant variables
print("Unique years: ", merged_data['Year'].unique())
print("*****")
print("Unique offices: ", merged_data['Office'].unique())
print("*****")


# we decide to drop these columns because they are all the same value
merged_data = merged_data.drop(columns=['Year', 'Office'])

print(merged_data.iloc[:, 0:6]) #not printing all columns for cleaner output


Size of dataset:  (1200, 21)
*****
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1200 entries, 0 to 1199
Data columns (total 21 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   County                                 1200 non-null   object 
 1   State                                  1200 non-null   object 
 2   Year                                   1200 non-null   int64  
 3   Office                                 1200 non-null   object 
 4   Democratic                             1200 non-null   float64
 5   Republican                             1200 non-null   float64
 6   FIPS                                   1188 non-null   float64
 7   Total Population                       1188 non-null   float64
 8   Citizen Voting-Age Population          1188 non-null   float64
 9   Percent White, not Hispanic or Latino  1188 non-null   float64
 10  Percent Black, not Hispanic or Latino

*** Task 3 Response *** 

The merged dataset has 21 variables, as shown at the top of the output. The County, State, and Office columns are strings, the year is a int64, and all other columns are float64. 

It is hard to tell if any of the demographic data is irrelevent at this point in the project, since all of it can be used to make meaningful comparisions based on income, population, age distribution, etc. The join used for Task 2 eliminated the duplicate County and State columns, and the only two irrelevent/redudant columns are the Year and Office ones. After analyzing their values they both only have one unique value (2018 for year, US Senator for office) and we dealt with them by dropping them to reduce the number of columns. This means our dataset now has 19 columns.

In [195]:
#Task 4: searching for missing values

print("Columns containing missing values and their counts:\n")
for c in merged_data:
    if merged_data[c].isnull().any():
        print('\t{0} has {1} null values'.format(c, merged_data[c].isnull().sum()))
        
        
# drop duplicates in the data        
merged_data = merged_data.drop_duplicates()
print("\nSize of dataset after dropping duplicates: ", merged_data.shape)

# drop rows containing missing data
merged_data = merged_data.dropna()
print("\nSize of dataset after dropping rows with missing values: ", merged_data.shape)

Columns containing missing values and their counts:

	FIPS has 12 null values
	Total Population has 12 null values
	Citizen Voting-Age Population has 12 null values
	Percent White, not Hispanic or Latino has 12 null values
	Percent Black, not Hispanic or Latino has 12 null values
	Percent Hispanic or Latino has 12 null values
	Percent Foreign Born has 12 null values
	Percent Female has 12 null values
	Percent Age 29 and Under has 12 null values
	Percent Age 65 and Older has 12 null values
	Median Household Income has 12 null values
	Percent Unemployed has 12 null values
	Percent Less than High School Degree has 12 null values
	Percent Less than Bachelor's Degree has 12 null values
	Percent Rural has 12 null values

Size of dataset after dropping duplicates:  (1200, 19)

Size of dataset after dropping rows with missing values:  (1188, 19)


*** Task 4 Response *** 

As shown above, the merged data is missing demographic information for 12 counties. Since the merged set has 1200 entries, we can afford to drop the 12 rows containing missing values and still be able to accurately interpret the data.

There were no duplicate values.


In [201]:
#Task 5: assigning a party to each county based on majority vote 
import numpy as np

# value of 'Party' is 1 if  # Democratic votes > # Republican votes, 0 otherwise
merged_data['Party'] = np.where(merged_data['Democratic'] > merged_data['Republican'] , 1, 0)

print(merged_data.loc[:, ['Democratic', 'Republican', 'Party']])
 

      Democratic  Republican  Party
0         3146.0      7511.0      0
1          364.0       796.0      0
2         3334.0      6487.0      0
3         2635.0      6000.0      0
4        14880.0     23419.0      0
...          ...         ...    ...
1195     51387.0     32849.0      1
1196      1281.0      3659.0      0
1197     69272.0     95814.0      0
1198       821.0      5543.0      0
1199      1392.0       821.0      1

[1188 rows x 3 columns]
